In [ ]:
##
#Import Packages
import numpy as np
import nibabel as nib
import sys
import matplotlib.pyplot as plt
import pickle
import torch

##
#SBI Specific Packages
from sbi import analysis as analysis
from sbi import utils as utils

In [ ]:
##
#Define Path to Code Database
DirPath = '/PATH/TO/bin/'

#Define Path for Posterior Object
InputPath = '/PATH/TO/Posterior.pkl'

#Define Path to Example Data
DataPath = '/PATH/TO/ExampleData/'

#Define Output Path
OutputPath = '/PATH/TO/Output/'

In [ ]:
##
#Import Custom Functions
sys.path.append(DirPath)
from ImportData import *
from FreedAnalytical import *

In [ ]:
##
#Load Posterior
with open(InputPath, "rb") as handle:
    posterior = pickle.load(handle)

In [ ]:
##
#Load Data
    # Data - DW-SSFP Data (4D)
    # T1Map - T1 Map (ms, 3D)
    # T2Map - T2 Map (ms, 3D)
    # B1Map - B1 Map (Normalised, 3D)
    # Mask - Mask (Binary, 3D)
    # noisefloor - noisefloor estimate (1xn)
    # bvecs - bvectors (3xn)
    # FlipAngles - Flip Angles (degrees) (1xn)
    # tau - Diffusion Gradient Duration (seconds) (1xn), 
    # G - Diffusion Gradient Duration (G/cm - Equivalent to mT/m Divided by 10) (1xn)
    # TRs - Repetition Times (seconds) (1xn)
    # b0s - Array Defining b0 Locations (b0 = 1, dwi = 0) (1xn)

Data, T1Map, T2Map, B1Map, Mask, noisefloor, bvecs, FlipAngles, tau, G, TRs, b0s = ImportDataDWSSFP(DataPath)

In [ ]:
##
#Estimate Mean Noisefloor
Noisefloor_Mean = np.mean(noisefloor)

In [ ]:
##
#Normalise Data by S0 (Estimate S0 per Flip Angle) & Perform Noisefloor Correction 

##
#Define Unique Locations
Values, Index = np.unique(FlipAngles, return_index=True)

##
#Estimate Normalised Data for each Flip Angle
for idx, k in enumerate(Index):
    #Calculate Theoretical Signal Amplitude
    b0 = FreedDWSSFP(G[k], tau[k], TRs[k], FlipAngles[k]*B1Map.data, 0, T1Map.data, T2Map.data)
    #Estimate S0 (Incorporating Noisefloor Contribution)
    S0 = np.abs(np.mean(Data.data[:,:,:,(b0s == 1) & (FlipAngles == Values[idx])],axis = 3)**2-Noisefloor_Mean**2)**0.5/b0*Mask.data
    #Divide by S0 & Remove NoiseFloor
    Data.data[:,:,:,FlipAngles == Values[idx]] = np.abs(Data.data[:,:,:,FlipAngles == Values[idx]]**2-Noisefloor_Mean**2)**0.5/S0[:,:,:,np.newaxis]

In [ ]:
##
#Obtain Posterior Distribution

##
#Define Number of Samples
Samples = 4500

##
#Pick Voxel (Note this is Not the Same Data Analysed in Figure 10 of the Manuscript, Here being Drawn from the Example Data)
k = 10; l = 13; m = 9 # Callosum

##
#Concatenate Data
DataVoxel = np.concatenate((Data.data[k,l,m,:],[B1Map.data[k,l,m]/100],[T1Map.data[k,l,m]/100000],[T2Map.data[k,l,m]/10000]))

##
#Obtain Posterior
SBI_Output = posterior.sample((Samples,), x=torch.from_numpy(DataVoxel[np.newaxis,:]), show_progress_bars = False)

In [ ]:
##
#Plot
SBIPlot = analysis.pairplot(SBI_Output*10000, limits=[[0, 3], [0, 3], [0, 3], [-1, 1], [-1, 1], [-1, 1]], labels = ([r'D$_{xx}$ ($\cdot 10^{-4}$ mm$^2$/s)',r'D$_{yy}$ ($\cdot 10^{-4}$ mm$^2$/s)',r'D$_{zz}$ ($\cdot 10^{-4}$ mm$^2$/s)',r'D$_{xy}$ ($\cdot 10^{-4}$ mm$^2$/s)',r'D$_{xz}$ ($\cdot 10^{-4}$ mm$^2$/s)',r'D$_{yz}$ ($\cdot 10^{-4}$ mm$^2$/s)']), figsize=(12, 12))

In [ ]:
##
#Save Figure
SBIPlot[0].savefig(''.join([OutputPath,'Figure10.pdf']),dpi=300,format='pdf',bbox_inches='tight')